In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam

2024-08-12 09:38:57.385487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 09:38:57.438687: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 09:38:57.453408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 09:38:57.566304: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 09:38:59.330597: W tensorflow/compiler/tf2

In [2]:
if tf.config.list_physical_devices('GPU'):
  print("GPU is available")
  print("CUDA Available: ", tf.config.list_physical_devices('GPU'))
  
else:
  print("GPU is not available")

GPU is available
CUDA Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1723444742.317597  318956 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723444742.645761  318956 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723444742.645835  318956 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
# Train data file location, file must be csv
train_path = "data_sets/sample_train.csv"
# Test data file location, file must be csv
test_path = "data_sets/sample_test.csv"

In [3]:
train_df = pd.read_csv(train_path)
if 'Unnamed: 0' in train_df.columns:
    train_df = train_df.drop('Unnamed: 0', axis=1)

test_df = pd.read_csv(test_path)
if 'Unnamed: 0' in test_df.columns:
    test_df = test_df.drop('Unnamed: 0', axis=1)

a
(547277, 20)


,Millis,ECG,IR,RED,patient,rate_avg,rate_std,rate_min,rate_max,result,sex,HT,DM,HL,KAH,KOAH,KBY,KC HST,height,weight
0,892,226.0,28556.0,28966.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
1,893,226.0,28554.0,28959.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
2,893,226.0,28556.0,28960.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
3,893,226.0,28552.0,28959.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
4,894,226.0,28551.0,28954.0,1,0.0,0.0,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0


In [ ]:
norm_cols = train_df.drop(columns=['Millis', 'patient', 'result', 'sex']).columns
scaler = StandardScaler()
train_df[norm_cols] = scaler.fit_transform(train_df[norm_cols])
test_df[norm_cols] = scaler.transform(test_df[norm_cols])
n_classes = len(np.unique(train_df.result))

In [ ]:
features = len(train_df.drop(columns=['result']).columns)

train_X = train_df.drop('result', axis=1)
train_y = train_df.result
test_X = test_df.drop('result', axis=1)
test_y = test_df.result

# Ensure train_y and test_y are binary (0 or 1)
train_y = train_y.astype(np.float32)
test_y = test_y.astype(np.float32)

train_X = train_X.values.reshape((train_X.values.shape[0], train_X.values.shape[1], 1))
test_X = test_X.values.reshape((test_X.values.shape[0], test_X.values.shape[1], 1))

In [ ]:
epochs=20

timesteps = 2048
batch_size=1024

learning_rate = 0.005

In [ ]:
model = Sequential()
model.add(GRU(250, input_shape=(timesteps, features), return_sequences=True))
model.add(GRU(250))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# Training the model
model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size)

# Predicting on the same data (for simplicity)
predictions = model.predict(test_X)
preds = (predictions > 0.5).astype(int)

print(classification_report(test_y, preds))

In [ ]:
test